In [58]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

year = 2023
df = pd.read_csv('./gaon_final_data.csv', index_col=0)
df = df.loc[df['year']==year]

# 노래제목 리스트
music_list = df['music'].unique().tolist()
music_list = music_list[:5]

# singer
for music in music_list:
    try:
        singer = df['singer'].loc[df['music']==music].tolist()[0]
    except:
        print('오류', music)


# 노래 id 딕셔너리 만들기 (key:노래id값, value:노래 제목)
id_dict={}
for idx, music in enumerate(music_list):
    

    id_dict[idx] = [music, singer]
    # time.sleep(1)


In [61]:
id_dict[0][0]

'Ditto'

# 멜론 최종 코드드

In [56]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

year = 2023
df = pd.read_csv('./gaon_final_data.csv', index_col=0)
df = df.loc[df['year']==year]

# 노래제목 리스트
music_list = df['music'].unique().tolist()
music_list = music_list[20:100]

# singer
for music in music_list:
    try:
        singer = df['singer'].loc[df['music']==music].tolist()[0]
    except:
        print('오류', music)


# 노래 id 딕셔너리 만들기 (key:노래id값, value:노래 제목)
id_dict={}
for music in music_list:
    singer = df['singer'].loc[df['music']==music].tolist()[0]

    # 가수이름 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if singer.find('(') != -1:
        singer = re.sub('\([^)]*\)','', singer) 
    else:
        pass

    # 노래제목 - 정규표현식으로 괄호 안의 내용 모두 삭제
    if music.find('(') != -1:
        music = re.sub('\([^)]*\)','', music) 
    else:
        pass
    
    singer = singer.rstrip()
    music = music.rstrip()

    # 공백을 '+'로 바꿔줌
    music = music.replace(' ', '+')
    singer = singer.replace(' ', '+')

    # 특수문자 제거
    singer = re.sub('[-=+#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",singer)

    print(f'music:{music}, singer:{singer}')

    try:
        URL = f'https://www.melon.com/search/total/index.htm?q={music}+{singer}'
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')

        # try:
        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        #frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input

        music_id = links.attrs['value']
        print(music_id)
    except:
        print(f'오류, music:{music}')
        pass

    # +을 공백로 바꿔줌
    music = music.replace('+', ' ')
    singer = singer.replace('+', ' ')

    id_dict[music_id] = [music, singer]
    time.sleep(1)

# 노래 id만 있는 리스트 
id_list = list(id_dict.keys())



# 데이터 뽑아오기
result = []

for id in id_list:
    music = id_dict[id][0]
    singer = id_dict[id][1]
    print(f'현재 {music} 크롤링 중')

    try:
        URL_1 = f'https://www.melon.com/song/detail.htm?songId={id}'
        response_1 = requests.get(URL_1,headers=header)
        time.sleep(1)
        soup = BeautifulSoup(response_1.text, 'html.parser')

        # 발매일, 장르
        release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
        category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
        

        # 좋아요 개수 json 접근
        URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={id}'
        response_2 = requests.get(URL_2,headers=header).text
        time.sleep(1)
        like_json = json.loads(response_2)
        music_like = like_json['contsLike'][0]['SUMMCNT']

        # 코멘트 개수 json 접근
        URL_3 = f'https://www.melon.com/song/songReviewCnt.json?songId={id}'
        response_3 = requests.get(URL_3,headers=header).text
        time.sleep(1)

        comment_json = json.loads(response_3)
        comment_count = comment_json['reviewValidCmtCnt']

        temp = {
        'music':music,
        'singer':singer,
        'music_id':id,
        'release_date':release_date,
        'category':category,
        'music_like':music_like,
        'comment_count':comment_count,
    }

        result.append(temp)
    except:
        print(f'오류, music:{music}')
        pass


df = pd.DataFrame(result)
df.to_csv(f'./melon_data_{year}.csv')



music:FEARLESS, singer:LESSERAFIM
35009233
music:그대를+알고, singer:송하예
35920493
music:Rush+Hour, singer:크러쉬
35665282
music:사랑인가+봐, singer:멜로망스
34657844
music:Pink+Venom, singer:BLACKPINK
35542908
music:잘가요, singer:주호
35890454
music:해요, singer:안녕
35252996
music:그때+그+순간+그대로, singer:WSG워너비
35383397
music:TOMBOY, singer:아이들
34754292
music:그라데이션, singer:10cm
35361345
music:딱+10CM만, singer:10cm,BIGNaughty
35668899
music:정이라고+하자, singer:BIGNaughty
34927767
music:나의+X에게, singer:경서
34943312
music:내가+아니라도, singer:주호
34787226
music:미운오리새끼, singer:허성현
35957694
music:자격지심, singer:BEO
35685439
music:취중고백, singer:김민석
34431086
music:오르트구름, singer:윤하
34270805
music:눈++), singer:던말릭
35937397
music:ELEVEN, singer:IVE
34349913
music:That`s+Hilarious, singer:CharliePuth
34864406
music:Say+I+Love+You, singer:우디
35953611
music:다정히+내+이름을+부르면, singer:경서예지,전건호
33496587
music:Snowman, singer:Sia
30717645
music:첫사랑, singer:백아
31343875
music:그댄+행복에+살텐데, singer:최유리
35958308
music:보고싶었어, singer:WSG워너비
35383398
music:모든

## 노래의 id값 먼저 뽑아오기

In [5]:
music_list

['Dancing Queen', '사랑은 이렇게', '되돌리다', '카페인', '매력있어']

In [5]:
import re

id_dict={}
for music in music_list:
    singer = df['singer'].loc[df['music']==music].tolist()[0]
  
    # 정규표현식으로 괄호 안의 내용 모두 삭제
    if singer.find('(') != -1:
        singer = re.sub(' \([^)]*\)','', singer) 
    # 괄호 없는 경우는 변수에 가수 이름 그대로 저장
    else:
        # singer_bracket = singer
        pass

    # 공백을 '+'로 바꿔줌
    music = music.replace(' ', '+')
    singer = singer.replace(' ', '+')


    URL = f'https://www.melon.com/search/total/index.htm?q={singer}+{music}'
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    music_id = links.attrs['value']

    # except:
    #     links = soup.select_one('#frm_searchSong > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    #     music_id = links.attrs['value']

    # finally:
        # '+'를 다시 공백으로 바꿔줌
    music = music.replace('+', ' ')
    singer = singer.replace('+', ' ')

    id_dict[music_id] = music
    time.sleep(1)


    

In [45]:
id_dict

{'35945927': 'OMG',
 '34819473': '사건의 지평선',
 '35454426': 'Hype boy',
 '35937396': 'NOT SORRY',
 '35931532': 'Candy',
 '35729104': 'ANTIFRAGILE',
 '35546497': 'After LIKE',
 '35454425': 'Attention',
 '35730562': 'Nxde',
 '35595136': '새삥',
 '35888871': '마이웨이',
 '34847378': 'LOVE DIVE',
 '35643794': 'Monologue',
 '4446485': '너의 모든 순간',
 '35485544': 'Cookie',
 '35849863': 'Dreamers',
 '35640077': 'Shut Down',
 '35876006': 'WHEN I MOVE',
 '34061322': '사랑은 늘 도망가',
 '35009233': 'FEARLESS',
 '35920493': '그대를 알고',
 '35665282': 'Rush Hour',
 '34657844': '사랑인가 봐',
 '35542908': 'Pink Venom',
 '35890454': '잘가요',
 '35252996': '해요',
 '35383397': '그때 그 순간 그대로',
 '34754292': 'TOMBOY',
 '35361345': '그라데이션',
 '35668899': '딱 10CM만',
 '34927767': '정이라고 하자',
 '34943312': '나의 X에게',
 '34787226': '내가 아니라도',
 '35957694': '미운오리새끼',
 '35685439': '자격지심',
 '34431086': '취중고백',
 '34270805': '오르트구름',
 '35937397': '눈  )',
 '34349913': 'ELEVEN',
 '34864406': 'That`s Hilarious',
 '35953611': 'Say I Love You',
 '33496587'

In [47]:
id_list = list(id_dict.keys())
id_list = id_list[:5]

In [54]:
result = []

for id in id_list:
    music = id_dict[id]

    URL_1 = f'https://www.melon.com/song/detail.htm?songId={id}'
    response_1 = requests.get(URL_1,headers=header)
    soup = BeautifulSoup(response_1.text, 'html.parser')

    # 발매일, 장르
    release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
    category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
    

    # 좋아요 개수 json 접근
    URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={id}'
    response_2 = requests.get(URL_2,headers=header).text

    like_json = json.loads(response_2)
    music_like = like_json['contsLike'][0]['SUMMCNT']

    # 코멘트 개수 json 접근
    URL_3 = f'https://www.melon.com/song/songReviewCnt.json?songId={id}'
    response_3 = requests.get(URL_3,headers=header).text

    comment_json = json.loads(response_3)
    comment_count = comment_json['reviewValidCmtCnt']


    print(f'id:{id}, release_date:{release_date}, category:{category} music_like:{music_like}, comment_count:{comment_count}')

    temp = {
    'music':music,
    'music_id':id,
    'release_date':release_date,
    'category':category,
    'music_like':music_like,
    'comment_count':comment_count,
}

    result.append(temp)

    df = pd.DataFrame(result)
    df
# df.to_csv(f'./melon_data_test2.csv')
# time.sleep(1)

id:35945927, release_date:2022.12.19, category:댄스 music_like:254724, comment_count:3086
id:34819473, release_date:2022.03.30, category:록/메탈 music_like:300079, comment_count:3148
id:35454426, release_date:2022.08.01, category:댄스 music_like:261042, comment_count:2052
id:35937396, release_date:2022.12.17, category:랩/힙합 music_like:94572, comment_count:609
id:35931532, release_date:2022.12.16, category:댄스 music_like:149063, comment_count:7251


In [43]:
id = '35937396'
# 좋아요 가져오기위해 json으로 접근
URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={id}'
response_2 = requests.get(URL_2,headers=header).text

like_json = json.loads(response_2)
music_like = like_json['contsLike'][0]['SUMMCNT']
music_like

94562

In [46]:
import re

id = '35945927'
# 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
try:
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    
    response = requests.get(URL_3,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
    movie = movie.attrs['href']
    movie = re.sub('[a-zA-z:.(\')]', '', movie)
    movie_id = movie.split(', ')[1]

    time.sleep(1)
    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
    URL_4 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={id}'
    response_3 = requests.get(URL_4, headers=header).text
    movie_json = json.loads(response_3)
    movie_like = movie_json['contsLike'][0]['LOVECNT']
    movie_views = movie_json['contsLike'][0]['READCNT']
    time.sleep(1)

except:
    print(f'뮤비 없음 id:', id)
    

뮤비 없음 id: 35945927


C:\Users\bin\AppData\Local\Temp\ipykernel_33324\1386413818.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
import re

id = '35945927'
# 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
try:
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    
    response = requests.get(URL_3,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
    movie = movie.attrs['href']
    movie = re.sub('[a-zA-z:.(\')]', '', movie)
    movie_id = movie.split(', ')[1]

    time.sleep(1)
    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
    URL_4 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={id}'
    response_3 = requests.get(URL_4, headers=header).text
    movie_json = json.loads(response_3)
    movie_like = movie_json['contsLike'][0]['LOVECNT']
    movie_views = movie_json['contsLike'][0]['READCNT']
    time.sleep(1)

except:
    print(f'뮤비 없음 id:', id)
    

In [50]:
id = '35945927'
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}

response = requests.get(URL_3,headers=header)
soup = BeautifulSoup(response.text, 'html.parser')

movie = soup.select('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li:nth-child(1) > div.thumb > a')

# movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
# movie = movie.attrs['href']
# movie = re.sub('[a-zA-z:.(\')]', '', movie)
# movie_id = movie.split(', ')[1]
print(movie)

[]


C:\Users\bin\AppData\Local\Temp\ipykernel_33324\1117603341.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')
